## Load a v0 QA pipeline
Follow the steps [here](https://haystack.deepset.ai/tutorials/01_basic_qa_pipeline)

In [6]:
# Import some files
import pickle
with open('./chapter_fmt_list.pkl','rb') as f:
    chapters = pickle.load(f)

In [7]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [8]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)


INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


In [10]:
# Write some documents to the store with their metadata
document_store.write_documents(documents=chapters)

Updating BM25 representation...: 100%|██████████| 307/307 [00:07<00:00, 41.99 docs/s] 


In [11]:
# Initialize a Retreiver
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


In [12]:
# initialize the Reader
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


In [13]:
# Create a retreiver-reader pipeline
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [26]:
# Ask some questions and get some answers
prediction = pipe.run(
    query="How many Carmine Conspirators are there?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

WARNING - haystack.document_stores.memory -  InMemoryDocumentStore does not support scale_score for BM25 retrieval. This parameter is ignored.
Inferencing Samples: 100%|██████████| 29/29 [01:21<00:00,  2.80s/ Batches]
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-14133, -14129) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-11561, -11556) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-5634, -5629) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-6867, -6862) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-2985, -2981) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-21780, -21774) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-14630, -14622) with a span answer. 
ERROR - haystack.modeling.model.predictions - 

In [27]:
# Print out some answers
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'three', 'type': 'extractive', 'score': 0.5941977500915527, 'context': '    The three of them stood in the main room of the basement.  Snowdrop had gone upstairs to keep an eye on Matthew and keep an ear out for trouble, w', 'offsets_in_document': [{'start': 8, 'end': 13}], 'offsets_in_context': [{'start': 8, 'end': 13}], 'document_id': '9d1d5be51f83bd238a4900ccc6cee6c4', 'meta': {'arc_title': 'Dash to Pieces', 'pov': 'Lucy', 'wordcount': '10736', 'series_chapter_number': 107, 'arc_number': '11', 'extra_material': False, 'title': 'Dash to Pieces - 11.4', 'chapter': '11.4'}}>,
             <Answer {'answer': 'ds,” A', 'type': 'extractive', 'score': 0.5874567031860352, 'context': '    Lucy’s leg bounced nervously as she waited.  Her earring didn’t catch any imminent approaches.  It was dark out, and the snow that blew around was', 'offsets_in_document': [{'start': -21898, 'end': -21892}], 'offsets_in_context': [{'start': -21898, 'end': -21892}], 'document_

In [28]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="all" ## Choose from `minimum`, `medium`, and `all`
)



Query: How many Carmine Conspirators are there?
Answers:
[   <Answer {'answer': 'three', 'type': 'extractive', 'score': 0.5941977500915527, 'context': '    The three of them stood in the main room of the basement.  Snowdrop had gone upstairs to keep an eye on Matthew and keep an ear out for trouble, w', 'offsets_in_document': [{'start': 8, 'end': 13}], 'offsets_in_context': [{'start': 8, 'end': 13}], 'document_id': '9d1d5be51f83bd238a4900ccc6cee6c4', 'meta': {'arc_title': 'Dash to Pieces', 'pov': 'Lucy', 'wordcount': '10736', 'series_chapter_number': 107, 'arc_number': '11', 'extra_material': False, 'title': 'Dash to Pieces - 11.4', 'chapter': '11.4'}}>,
    <Answer {'answer': 'ds,” A', 'type': 'extractive', 'score': 0.5874567031860352, 'context': '    Lucy’s leg bounced nervously as she waited.  Her earring didn’t catch any imminent approaches.  It was dark out, and the snow that blew around was', 'offsets_in_document': [{'start': -21898, 'end': -21892}], 'offsets_in_context': [{'sta